создаем файл с числами

In [19]:
import random

# Открываем файл для записи
with open("random_numbers.txt", "w") as file:
    # Генерируем и записываем 50000 случайных 32-битных целых чисел
    for _ in range(50000):
        random_number = random.randint(2, 2**31 - 1)
        file.write(str(random_number) + "\n")

простой последовательный алгоритм 

In [20]:
import math
import time
# Функция для подсчета простых множителей числа

def count_prime_factors(num):
    count = 0
    # Проверяем делители 
    for i in range(2, int(math.sqrt(num)) + 1):
        while num % i == 0:
            count += 1
            num //= i
    return count + (num > 1)

start_time = time.time()

# Считываем числа из файла и подсчитываем простые множители для каждого числа
total_prime_factors = 0

with open("random_numbers.txt", "r") as file:
    for line in file:
        number = int(line.strip())
        total_prime_factors += count_prime_factors(number)

end_time = time.time()
res = end_time - start_time
print("Суммарное количество простых множителей для чисел из файла:", total_prime_factors)
print(f"Время выполнения {res:.1f} секунд")

Суммарное количество простых множителей для чисел из файла: 203380
Время выполнения 85.9 секунд


мультипроцессинг 

In [21]:
import math
import multiprocessing
import time

start_time = time.time()

# Функция для подсчета простых множителей числа
def count_prime_factors(num):
    count = 0
    # Проверяем делители 
    for i in range(2, int(math.sqrt(num)) + 1):
        while num % i == 0:
            count += 1
            num //= i
    return count + (num > 1)

# Функция для обработки данных в отдельном процессе
def process_data(chunk):
    total_prime_factors = sum(count_prime_factors(num) for num in chunk)
    return total_prime_factors

# Открываем файл для чтения и считываем числа
numbers = []
with open("random_numbers.txt", "r") as file:
    for line in file:
        numbers.append(int(line.strip()))

# Подготовка данных для распределения по процессам
chunk_size = len(numbers) // multiprocessing.cpu_count()
chunks = [numbers[i:i + chunk_size] for i in range(0, len(numbers), chunk_size)]

# Создание процессов и запуск обработки данных
results = []
for chunk in chunks:
    result = process_data(chunk)
    results.append(result)



# Суммируем результаты из всех процессов
total_prime_factors = sum(results)

end_time = time.time()
res = end_time - start_time

print("Суммарное количество простых множителей для чисел из файла:", total_prime_factors)
print(f"Время выполнения {res:.1f} секунд")


Суммарное количество простых множителей для чисел из файла: 203380
Время выполнения 76.8 секунд


с помощью Dask

In [22]:
import math
import dask.bag as db
import time

start_time = time.time()
# Функция для подсчета простых множителей числа
def count_prime_factors(num):
    count = 0
    # Проверяем делители 
    for i in range(2, int(math.sqrt(num)) + 1):
        while num % i == 0:
            count += 1
            num //= i
    return count + (num > 1)

# Чтение чисел из файла 
numbers_bag = db.read_text("random_numbers.txt").map(int)

# Подсчет простых множителей для каждого числа
prime_factors_count = numbers_bag.map(count_prime_factors).sum().compute()


end_time = time.time()
res = end_time - start_time
print("Суммарное количество простых множителей для чисел из файла:", prime_factors_count)
print(f"Время выполнения {res:.1f} секунд")

Суммарное количество простых множителей для чисел из файла: 203380
Время выполнения 79.0 секунд
